NLP Tasks and Applications

fine-tuning은 단순히 특정 도메인에 특화된 LM을 만드는것 이상의 유용성을 지님\
GLUE, SQuAD\
NER, text classification

대표적인 NLP task\
sequence classification : 주어진 시퀀스를 특정 클래스에 분류, 2장에서 다룬 감성분석, 문장 간 관계 파악(가설과 주어진 문장의 사실과 일치, 중립, 모순)\
QA : 텍스트, 음성 데이터 질문에 대해 기계가 문서에서 정답이 포함된 문장 찾아내 출력\
Language Modeling : 주어진 단어 시퀀스를 기반으로 다음 단어 예측. 앞에서부터 순차적으로 다음 단어 예측, 문장에서 일부 단어를 가리고 해당 단어 예측.\
Text generation : 긴 문장, 문단을 자연스럽게 생성\
NER : 문장에서 특정 단어를 특정 개체(사람, 국가, 지역, 조직 등)로 분류\
Summarization\
Machine Translation\

NER 과 text classification을 직접 해보자


In [9]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("amananandrai/ag-news-classification-dataset")

print("Path to dataset files:", path)

Path to dataset files: /home/ajh1729/.cache/kagglehub/datasets/amananandrai/ag-news-classification-dataset/versions/2


In [1]:
# Import libraries
import pandas as pd
import os

# Get current working directory
# cwd = os.getcwd()

# Import AG Dataset
# 뉴스 기사 데이터셋셋
data = pd.read_csv('/home/ajh1729/.cache/kagglehub/datasets/amananandrai/ag-news-classification-dataset/versions/2/train.csv')
data = pd.DataFrame(data=data)
data.columns = data.columns.str.replace(" ","_")
data.columns = data.columns.str.lower()
data["class_name"] = data["class_index"].map({1:"World", 2:"Sports", 3:"Business", 4:"Sci_Tech"})

In [2]:
data

,class_index,title,description,class_name
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli...",Business
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...,Business
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...,Business
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...,Business
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco...",Business
...,...,...,...,...
119995,1,Pakistan's Musharraf Says Won't Quit as Army C...,KARACHI (Reuters) - Pakistani President Perve...,World
119996,2,Renteria signing a top-shelf deal,Red Sox general manager Theo Epstein acknowled...,Sports
119997,2,Saban not going to Dolphins yet,The Miami Dolphins will put their courtship of...,Sports
119998,2,Today's NFL games,PITTSBURGH at NY GIANTS Time: 1:30 p.m. Line: ...,Sports


In [3]:
# View titles
for i in range(10):
    print("Title of Article",i)
    print(data.loc[i,"title"])
    print("\n")

Title of Article 0
Wall St. Bears Claw Back Into the Black (Reuters)


Title of Article 1
Carlyle Looks Toward Commercial Aerospace (Reuters)


Title of Article 2
Oil and Economy Cloud Stocks' Outlook (Reuters)


Title of Article 3
Iraq Halts Oil Exports from Main Southern Pipeline (Reuters)


Title of Article 4
Oil prices soar to all-time record, posing new menace to US economy (AFP)


Title of Article 5
Stocks End Up, But Near Year Lows (Reuters)


Title of Article 6
Money Funds Fell in Latest Week (AP)


Title of Article 7
Fed minutes show dissent over inflation (USATODAY.com)


Title of Article 8
Safety Net (Forbes.com)


Title of Article 9
Wall St. Bears Claw Back Into the Black




In [4]:
# View descriptions
for i in range(10):
    print("Description of Article",i)
    print(data.loc[i,"description"])
    print("\n")

Description of Article 0
Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.


Description of Article 1
Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.


Description of Article 2
Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.


Description of Article 3
Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.


Description of Article 4
AFP - Tearaway world oil prices, toppling records and straining wallets, present a new economic menace barely three months before the US presidential elections.


Description o

In [5]:
# Clean up text
cols = ["title","description"]
data[cols] = data[cols].applymap(lambda x: x.replace("\\"," "))
data[cols] = data[cols].applymap(lambda x: x.replace("#36;","$"))
data[cols] = data[cols].applymap(lambda x: x.replace("  "," "))
data[cols] = data[cols].applymap(lambda x: x.strip())
# Write data to CSV
data.to_csv('/home/ajh1729/.cache/kagglehub/datasets/amananandrai/ag-news-classification-dataset/versions/2/prepared/train_prepared.csv', index=False)

NLP Task #1: Named Entity Recognition

In [12]:
import os
import torch
import spacy

# CUDA_VISIBLE_DEVICES 강제 적용
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# PyTorch에서 GPU 1번 강제 사용
torch.cuda.set_device(0)  # "cuda:0"은 CUDA_VISIBLE_DEVICES 설정을 따른 값임

# 현재 선택된 GPU 확인
print("✅ PyTorch가 선택한 GPU Index:", torch.cuda.current_device())
print("✅ 현재 사용 중인 GPU:", torch.cuda.get_device_name(torch.cuda.current_device()))

# spaCy에서 GPU 강제 사용 설정
spacy.require_gpu()
print("✅ spaCy GPU 사용 가능:", spacy.prefer_gpu())

# Transformer 기반 개체명 인식(NER) 모델 로드
nlp = spacy.load("en_core_web_trf")

print("✅ spaCy 모델 로드 성공! GPU 1번 사용 중 🚀")


✅ PyTorch가 선택한 GPU Index: 0
✅ 현재 사용 중인 GPU: NVIDIA GeForce RTX 3090
✅ spaCy GPU 사용 가능: True
✅ spaCy 모델 로드 성공! GPU 1번 사용 중 🚀


In [13]:
torch.cuda.current_device()

0

In [8]:
# Import spacy and load language model
import spacy

spacy.require_gpu()
print(spacy.require_gpu())
nlp = spacy.load("en_core_web_trf") #spacy에서 제공하는 사전학습된 NER 모델

True


In [14]:
# View metadata of the model
import pprint
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(nlp.meta)

{   'author': 'Explosion',
    'components': [   'transformer',
                      'tagger',
                      'parser',
                      'attribute_ruler',
                      'lemmatizer',
                      'ner'],
    'description': 'English transformer pipeline '
                   "(Transformer(name='roberta-base', "
                   "piece_encoder='byte-bpe', stride=104, type='roberta', "
                   'width=768, window=144, vocab_size=50265)). Components: '
                   'transformer, tagger, parser, ner, attribute_ruler, '
                   'lemmatizer.',
    'disabled': [],
    'email': 'contact@explosion.ai',
    'labels': {   'attribute_ruler': [],
                  'lemmatizer': [],
                  'ner': [   'CARDINAL',
                             'DATE',
                             'EVENT',
                             'FAC',
                             'GPE',
                             'LANGUAGE',
                             'LAW',

In [16]:
# Print NER results for Descriptions
for i in range(9):
    print("Article",i)
    print(data.loc[i,"description"])
    print("Text Start End Label")
    doc = nlp(data.loc[i,"description"])
    for token in doc.ents:
        print(token.text, token.start_char,
            token.end_char, token.label_)
    print("\n")

Article 0
Reuters - Short-sellers, Wall Street's dwindling band of ultra-cynics, are seeing green again.
Text Start End Label
Reuters 0 7 ORG


Article 1
Reuters - Private investment firm Carlyle Group, which has a reputation for making well-timed and occasionally controversial plays in the defense industry, has quietly placed its bets on another part of the market.
Text Start End Label
Reuters 0 7 ORG
Carlyle Group 34 47 ORG


Article 2
Reuters - Soaring crude prices plus worries about the economy and the outlook for earnings are expected to hang over the stock market next week during the depth of the summer doldrums.
Text Start End Label
Reuters 0 7 ORG
next week 134 143 DATE
summer 168 174 DATE


Article 3
Reuters - Authorities have halted oil export flows from the main pipeline in southern Iraq after intelligence showed a rebel militia could strike infrastructure, an oil official said on Saturday.
Text Start End Label
Reuters 0 7 ORG
Iraq 86 90 GPE
Saturday 186 194 DATE


Article 4

Custom NER\
spacy tfm 모델은 블로그, 뉴스, 댓글 데이터로 학습됨. 법률, 금융, 의료 등 전문적인 데이터를 다룬다면 모델이 데이터 이해 못 할 가능성 높음\
일부 데이터에 직접 주석을 달거나 이를 이용해 fine-tuning\

데이터에 주석을 달기 위해 prodigy라는 도구를 이용

In [17]:
# prodigy 사용법은 skip..

NER 모델 학습
- Transformer 기반 NER 모델
- CPU 기반 NER 모델


In [ ]:
a